![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [2]:
# Importando bibliotecas
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from timeit import timeit

In [3]:
# Definindo caminho dos dados
filename_features = "../Mod_16/data_tarefa_1/UCI HAR Dataset/features.txt"
filename_labels = "../Mod_16/data_tarefa_1/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "../Mod_16/data_tarefa_1/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "../Mod_16/data_tarefa_1/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "../Mod_16/data_tarefa_1/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "../Mod_16/data_tarefa_1/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "../Mod_16/data_tarefa_1/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "../Mod_16/data_tarefa_1/UCI HAR Dataset/test/y_test.txt"

# Lendo dados
features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")
labels = pd.read_csv(filename_labels, sep=r'\s+', header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
X_train = pd.read_csv(filename_xtrain, sep=r'\s+', header=None, names=features['nome_var'].tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
X_test = pd.read_csv(ffilename_xtest, sep=r'\s+', header=None, names=features['nome_var'].tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

# Repartindo base de dados para criar base de validação
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=.4)

In [4]:
# Visualizando tamanho das bases 
print(f'Tamanho base de treino: {X_train.shape}')
print(f'Tamanho base de validação: {X_valid.shape}')
print(f'Tamanho base de teste: {X_test.shape}')

Tamanho base de treino: (4411, 561)
Tamanho base de validação: (2941, 561)
Tamanho base de teste: (2947, 561)


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [6]:
%%time

# Criando objeto de árvore de classificação
clf = DecisionTreeClassifier(random_state=412, ccp_alpha=.001)

# Treinando o modelo
trained_model = clf.fit(X_train, y_train)

# Avaliando o modelo nas 3 bases
train_score = cross_val_score(trained_model, X_train, y_train, cv=5, scoring='accuracy')
valid_score = cross_val_score(trained_model, X_valid, y_valid, cv=5, scoring='accuracy')
test_score = cross_val_score(trained_model, X_test, y_test, cv=5, scoring='accuracy')

# Visualizando melhor acurácia em cada base
print(f'Melhor acurácia na base de treino: {train_score.max()*100:.2f}%')
print(f'Melhor acurácia na base de validação: {valid_score.max()*100:.2f}%')
print(f'Melhor acurácia na base de teste: {test_score.max()*100:.2f}%')

Melhor acurácia na base de treino: 93.20%
Melhor acurácia na base de validação: 93.20%
Melhor acurácia na base de teste: 87.95%
CPU times: total: 22.1 s
Wall time: 22.1 s


### Observações
O algoritmo acima apresentou um tempo de execução considerável para processar todas as árvores, alcançando uma acurácia de aproximadamente 88% na base de teste. No entanto, em uma aplicação real, essa demora pode impactar significativamente o fluxo do sistema em que o modelo será integrado. Caso o desempenho do modelo cause latência elevada, isso poderá rapidamente se tornar um problema crítico, comprometendo a eficiência e a experiência do usuário.

## Árvore com PCA

Faça uma análise de componentes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [7]:
# Função para rodar análise de componentes principais
def run_pca(estimator, n_components, *args):
    # Extração dos parâmetros
    pc_train, pc_valid, pc_test, y_train, y_valid, y_test = args
    # Definindo nome das colunas
    cols = ['cp'+str(x+1) for x in list(range(n_components))]

    # Selecionando componentes para treinar e avaliar o modelo
    pc_train_selected = pd.DataFrame(pc_train[:,:n_components], columns=cols)
    pc_valid_selected = pd.DataFrame(pc_valid[:,:n_components], columns=cols)
    pc_test_selected = pd.DataFrame(pc_test[:,:n_components], columns=cols)

    # Treinando o modelo
    trained_model_pca = estimator.fit(pc_train, y_train)

    # Avaliando o modelo nas bases
    train_score_pca = cross_val_score(trained_model_pca, pc_train_selected, y_train, cv=5, scoring='accuracy')
    valid_score_pca = cross_val_score(trained_model_pca, pc_valid_selected, y_valid, cv=5, scoring='accuracy')
    test_score_pca = cross_val_score(trained_model_pca, pc_test_selected, y_test, cv=5, scoring='accuracy')

    return train_score_pca, valid_score_pca, test_score_pca

In [9]:
%%time

# Criando objeto PCA e treinando-o
prcomp = PCA().fit(X_train)

# Criando matrizes de autovalores
pc_train = prcomp.transform(X_train)
pc_valid = prcomp.transform(X_valid)
pc_test = prcomp.transform(X_test)

# Rodando PCA com as matrizes criadas
train_score_pca, valid_score_pca, test_score_pca = run_pca(clf
                                                           ,1
                                                           ,pc_train
                                                           ,pc_valid
                                                           ,pc_test
                                                           ,y_train
                                                           ,y_valid
                                                           ,y_test)

# Visualizando melhor acurácia em cada base
print(f'Melhor acurácia na base de treino: {train_score_pca.max()*100:.2f}%')
print(f'Melhor acurácia na base de validação: {valid_score_pca.max()*100:.2f}%')
print(f'Melhor acurácia na base de teste: {test_score_pca.max()*100:.2f}%')

Melhor acurácia na base de treino: 49.38%
Melhor acurácia na base de validação: 49.92%
Melhor acurácia na base de teste: 49.83%
CPU times: total: 5.31 s
Wall time: 3.12 s


Como é possível observar acima, utilizando apenas um componente a acurácia na base de testes caiu consideravelmente, de 88% para 50%, no entanto, o tempo de execução do algoritmo também reduziu de forma considerável.

## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [10]:
%%time

# Criando dicionário para armezenar melhores acurácias e tempo de execução
scores = dict()

# Iterando o número de componentes
for n in [1, 2, 5, 10, 50]:
    # Rodando PCA e avaliando nas três bases
    tr_s, vl_s, ts_s = run_pca(clf, n, pc_train, pc_valid, pc_test, y_train, y_valid, y_test)
    # Coletando tempo de execução do algoritmo
    execution_time = timeit(stmt='run_pca(clf, n, pc_train, pc_valid, pc_test, y_train, y_valid, y_test)', globals=globals(),  number=1)
    # Salvando dados no dicionário
    scores[f'{n}_components'] = [tr_s.max(), vl_s.max(), ts_s.max(), execution_time]

# Visualizando melhor acurácia na base de teste com tempo de execução por número de componentes
for n in scores:
    print(f'Para um modelo PCA com {n}, a acurácia na base de teste é igual a {scores[n][2]*100:.2f}% e o tempo de execução é igual a {scores[n][-1]:.2f} segundos.')

Para um modelo PCA com 1_components, a acurácia na base de teste é igual a 49.83% e o tempo de execução é igual a 2.71 segundos.
Para um modelo PCA com 2_components, a acurácia na base de teste é igual a 61.97% e o tempo de execução é igual a 2.77 segundos.
Para um modelo PCA com 5_components, a acurácia na base de teste é igual a 77.25% e o tempo de execução é igual a 2.82 segundos.
Para um modelo PCA com 10_components, a acurácia na base de teste é igual a 84.24% e o tempo de execução é igual a 3.05 segundos.
Para um modelo PCA com 50_components, a acurácia na base de teste é igual a 84.89% e o tempo de execução é igual a 4.75 segundos.
CPU times: total: 32.2 s
Wall time: 32.2 s


Como é possível observar, conforme o número de componentes utilizados aumenta, aumenta-se consideravelmente a acurácia do modelo e o tempo de execução do mesmo permanece relativamente pequeno, comparado aos testes anteriores. 

## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

Conforme o número de componentes utilizados **aumenta**, a acurácia do modelo sofre uma melhora significativa, enquanto o tempo de processamento permanece relativamente baixo. Portanto, a abordagem de modelagem com PCA se prova melhor para implementação que lidam com muitas variáveis e requerem um tempo otimizado de execução.